# Building an ETL Pipeline for WRC Hackathon

### Tools

In [52]:
# Importing Packages

import tweepy as tw         
import pandas as pd     
import numpy as np
from datetime import date, timedelta
import json
import re
from IPython.display import display
import pyodbc

In [53]:
# API Keys

CONSUMER_KEY    = 'egoItCf7bZMp4gopTCXC0WH2O'
CONSUMER_SECRET = 'Ph7fyNWzMnLTb0rgx5w0GPb0bzWlfyUtfVhZZGt5jVk4FJeBxK'

ACCESS_TOKEN  = '840149748354965504-6fGpkvdj6n53uVG5231Oq6PhyLzHlfO'
ACCESS_SECRET = 'f17t2HIfmpsgh1IBxgdugigEH8Xuzhps7gjGT2jfLOgxT'

In [54]:
# Create authentication

auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tw.API(auth, wait_on_rate_limit=True)

### Data

Both sets need expansion

In [55]:
 key_words = [ 'leaks' , 'burst' , 'water pressure', 'water','cuts', 'pipe',
 'fault', 'broken', 'damaged', 'toilets', 'sanitation',
 'leakage', 'wasted', 'complaints', 'complained', 'reported', 'water supply',
 'service delivery', 'outage', 'no water', 'reference number', 'faults'
 'faults' ,'leaks' , 'report' , 'complain' ,'install']

In [56]:
master_list =[['alfred_enzo' ,'@alfrednzoDM', 'eastern_cape'],
              ['amajuba' ,'@AmajubaDM' , 'kwazulu_natal' ],
              ['cape_wine_lands','@CWDM2','western_cape'],
              ['ehlanzeni' ,'@EhlanzeniM','mpumalanga'],
              ['fezile_dabi', '@FezileDabiDM','free_state' ],
              ['garden_route', '@GardenRoute_DM','western_cape'],
              ['gert_sibande','@GertSibandeDM','mpumalanga' ],
              ['ilembe' ,'@DistrictIlembe','kwazulu_natal'],
              ['king_cetshwayo','@Cetshwayo_dm','kwazulu_natal'],
              ['lejweleputswa', '@LejweleputswaM','free_state' ],
              ['mangaung', '@MangaungCity','free_state' ],
              ['nkangala', '@NkangalaDM','mpumalanga' ],
              ['sedibeng', '@SedibengDM','gauteng' ],
              ['ugu', '@UguDM','kwazulu_natal'],
              ['north_uthungulu','@UmkhanyakudeM','kwazulu_natal'],
              ['uthukela', '@OkhahlambaLM','kwazulu_natal'],
              ['vhembe', '@VhembeDM','limpopo'],
              ['rand_west_city', '@RandLocal','gauteng' ],
              ['sedibeng' ,'@EmfuleniLM','gauteng' ],
              ['amathole','Amathole District', 'eastern_cape'] , 
              [ 'bojanala','Bojanala District', 'north_west'], 
              [ 'capricon' ,'Capricorn District', 'limpopo'], 
              ['karoo' ,'Central Karoo District', 'western_cape'],
              ['chris_hani' ,'Chris Hani District', 'eastern_cape'], 
              ['kenneth_kaunda','Dr Kenneth Kaunda District', 'north_west'], 
              ['dr_ruth','Dr Ruth Segomotsi Mompati District', 'north_west'] ,
              ['frances','Frances Baard District', 'northern_cape'],
              ['harry_gwala','Harry Gwala District', 'kwazulu_natal'], 
              ['joe_gqabi','Joe Gqabi District', 'eastern_cape' ],
              ['john_taolo','John Taolo Gaetsewe District', 'northern_cape'], 
              ['mopani','Mopani District', 'limpopo'],
              ['namakwa','Namakwa District', 'northern_Cape'], 
              ['ngaka','Ngaka Modiri Molema', 'north_west'], 
              ['or_tambo' ,'OR Tambo District', 'eastern_cape'], 
              ['overberge','Overberg District', 'western_cape'],
              ['pixley_ka_seme','Pixley ka Seme District', 'northern_cape'], 
              ['sarah', 'Sarah Baartman District', 'eastern_cape'],
              ['sekhukune','Sekhukhune District', 'limpopo'],
              ['thabo','Thabo Mofutsanyana District', 'free_state'] ,
              ['umgungundlovu' ,'uMgungundlovu District', 'kwazulu_natal'] ,
              ['umzinyathi','uMzinyathi District', 'kwazulu_natal'],
              ['waterberg','Waterberg District', 'limpopo' ],
              ['west_cost' ,'West Coast District', 'western_cape'], 
              ['zf_mchunu' ,'ZF Mgcawu District', 'northern_cape'],
              ['xhariep' ,'Xhariep District', 'free_state'],
              ['zululand' ,'Zululand District', 'kwazulu_natal']]

### Functions

In [57]:
def extract_tweets(search_word) :

    search_words = search_word + " -filter:retweets"
    today = date.today()                     
    tweets = tw.Cursor(api.search,
               q=search_words,
               lang="en"
               ).items(200)                    
    tweet_status = [tweet for tweet in tweets]
    tweet_list = [tweet.text for tweet in tweet_status]
    tweet_date = [tweet.created_at for tweet in tweet_status]
    co_od = [tweet.coordinates for tweet in tweet_status]

    tweet_df = pd.DataFrame(data=tweet_list, 
        columns=['tweet'])
    tweet_df['timestamp'] = tweet_date
    tweet_df['coordinates'] = co_od
        
    return tweet_df

In [58]:
def TweetCleaner(tweet):

    tweet = tweet.lower()  
    tweet = re.sub('@[\w]*','',tweet)  
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    tweet = re.sub(r'#\w*', '', tweet)     
    tweet = re.sub(r"[,.;':@#?!\&/$]+\ *", ' ', tweet)
    tweet = re.sub(r"U+FFFD ", ' ', tweet)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    tweet = tweet.lstrip(' ')
    tweet = ' '.join([i for i in tweet if i in key_words])
    
    return tweet

In [59]:
def create_df(the_list) :
    
    col_order = ['tweet', 'timestamp','keywords' ,'coordinates']
    data = [['a', 'b' ,'c' ,'d']]    
    dummy_df = pd.DataFrame(data, columns = ['tweet', 'timestamp','keywords' ,'coordinates'])

    for i in the_list :
        handle = i[1]
        df = extract_tweets(handle)
        df['keywords'] = df['tweet'].apply(TweetCleaner)
        df = df[col_order]

        dummy_df = dummy_df.append(df)
    
    dummy_df = dummy_df.drop([0])
    dummy_df = dummy_df.reset_index()
    dummy_df = dummy_df.drop(columns = ['index'] , axis = 1)   
    
    return dummy_df


### Build Data

In [ ]:
database = create_df(master_list)

In [ ]:
database.head()

### Database connection